In [27]:
import torch

In [28]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

# Retrieve pickle and csv data

In [29]:
#Run strucuture before retrieving pickle

class DfSubject:
    def __init__(self, elec_i, muscle_i):
        self.Elec_config=list_elecs[elec_i]
        # self.List_elecs=list_elecs # 0,1,2
        self.Muscle_or_other=muscle_i
        # self.List_muscle_or_other=list_muscle_or_other_elec_i
        self.Trial_number= df_subject[(df_subject["Elec_config"]==self.Elec_config)& (df_subject["Muscle"]== self.Muscle_or_other)]["index"]
        self.Trial_number=[int(i) for i in self.Trial_number] #save as int
        self.Filtered= df_subject[(df_subject["Elec_config"]==self.Elec_config)& (df_subject["Muscle"]== self.Muscle_or_other)]["EMGs_filt"]
        self.All_peak_peak=df_subject[(df_subject["Elec_config"]==self.Elec_config)& (df_subject["Muscle"]== self.Muscle_or_other)]["All_Peak2Peak"]
        # save the reps for the different trials
        a=df_subject[(df_subject["Elec_config"]==self.Elec_config)& (df_subject["Muscle"]== self.Muscle_or_other)]["index"]
        a.index=np.arange(0, len(a))
        b=[]
        for number in range (len(a)):
            b.append(reps[a[number]]) #reps where extracted before
        
        #Set all trials to 1000TP
        self.Time_points= [1000]*len(self.Trial_number)
    
        self.Amplitudes= [round(i,1) for i in df_subject[(df_subject["Elec_config"]==self.Elec_config)& (df_subject["Muscle"]== self.Muscle_or_other)]["Amplitudes"]] #round amplitudes
        self.Amplitudes=pd.Series(data=self.Amplitudes, index=np.arange(0, len(self.Amplitudes))) #convert amplitudes back to series
        # self.Amplitudes=[round(i, 1) for i in self.Amplitudes]
        # self.Amplitudes.index=np.arange(0, len(self.Amplitudes))
        self.Filtered.index=self.Amplitudes #index the emg series with the amplitude
        self.All_peak_peak.index=self.Amplitudes
        self.Reps=pd.Series(data=b, index=self.Amplitudes) #index the reps with the amplitude
     

In [30]:
### retrieve  dfSubject structured as pData  #
#run the structure of the data before opening the pickle (wihtout creating an object)
# command pData["_0_"].Trial_number=0
import pickle
run=1
if run == 1:  

    with open('/Users/nealarohner/Desktop/Projectome_Finder/code/Neala_Master_Thesis/data/pickle_files/filteredData_dump.pkl', 'rb') as handle:
        filteredData = pickle.load(handle)

In [31]:
# to retrieve ##
mt_electrodes = pd.read_csv('/Users/nealarohner/Desktop/Projectome_Finder/code/Neala_Master_Thesis/data/pickle_files/mt_electrodes.csv')
mt_muscles= pd.read_csv("/Users/nealarohner/Desktop/Projectome_Finder/code/Neala_Master_Thesis/data/pickle_files/mt_muscles.csv")

# Create tensor

## Modify data

In [32]:
device =  "cuda" if torch.cuda.is_available() else "cpu"

In [33]:
## n_Ampl_unique
## find all the amplitudes used(unique); to have one dim of amplitudes, where there is no trial for an amplitude, fill with nan

unique_amplitudes=[]
for elec_i in mt_electrodes['elec_name'] : #loop through all the elecs
    for muscle_i in mt_muscles['muscle_name']: # loop through all the muscles
            unique_amplitudes.extend(filteredData[f'_{elec_i}_{muscle_i}'].Amplitudes)


unique_amplitudes=np.unique(unique_amplitudes)  
print(unique_amplitudes)     

[0.2 0.4 0.5 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8 1.9 2.  2.1
 2.2 2.3 2.4 2.5 2.6 2.7 2.8 2.9 3.  3.1 3.2 3.3 3.4 3.5 3.6 3.7 3.8 4.
 4.5 4.7 4.8 4.9]


## Create filtered data tensor

In [80]:
#electrodes
#muscles
#amplitudes
#max_reps
#filtered EMG
max_reps=39 # see notebook 6
time_points=1000



filtered_data_to_tensor=torch.ones(len(mt_electrodes['elec_name']),len(mt_muscles['muscle_name']),len(unique_amplitudes),max_reps,time_points)
filtered_data_to_tensor.size()

torch.Size([16, 16, 40, 39, 1000])

fill tensor

In [81]:
## filling the 11 rows (repetition) for the same amp with the raw data/ resp nan value wheb there is no data for that amplitude
for elec_i in range (len(mt_electrodes['elec_name'])) : #loop through all the elecs
    for muscle_i in range (len(mt_muscles['muscle_name'])): # loop through all the muscles
        amp_index=0
        
        for amp_in_unique_amp in unique_amplitudes: # loop through all the amplitudes
            reps=0
            if amp_index< len(filteredData[f"_{elec_i}_{mt_muscles['muscle_name'][muscle_i]}"].Amplitudes):
                #Check for which amplitudes there is data
                if filteredData[f"_{elec_i}_{mt_muscles['muscle_name'][muscle_i]}"].Amplitudes[amp_index]==amp_in_unique_amp:
                    print("found a suiting amplitude")

                    #Fill all the reps with data
                    while reps<filteredData[f"_{elec_i}_{mt_muscles['muscle_name'][muscle_i]}"].Reps[amp_in_unique_amp]:
                        print("going to the while loop")
                        print(f"elec_i {elec_i},  muscle_i, {muscle_i}, reps: {reps}, amp_in_unique_amp, {amp_in_unique_amp}")
                        filtered_data_to_tensor[elec_i, muscle_i, amp_index,reps,:]=torch.tensor(filteredData[f'_{elec_i}_{mt_muscles["muscle_name"][muscle_i]}'].Filtered[amp_in_unique_amp][reps])
                        reps+=1

                    # Fill leftover reps with nan
                    else: 
                        print("finished the while loop")
                        print(f"elec_i {elec_i},  muscle_i, {muscle_i}, reps: {reps}, amp_in_unique_amp, {amp_in_unique_amp}")
                        if filteredData[f'_{elec_i}_{mt_muscles["muscle_name"][muscle_i]}'].Reps[amp_in_unique_amp]<max_reps:
                            filtered_data_to_tensor[elec_i, muscle_i, amp_index,reps:,:]=torch.zeros((len(filtered_data_to_tensor[elec_i, muscle_i, amp_index,reps:,0]),time_points)).float()
                            filtered_data_to_tensor[filtered_data_to_tensor==0]= float('nan') #didn't find torch.nan function
                    #increase the amplitude index by one
                    amp_index+=1
                #Fill the amplitudes without data with 'nan'
            else:
                filtered_data_to_tensor[elec_i, muscle_i, amp_index,:,:]=torch.zeros((max_reps,time_points)).float()
                filtered_data_to_tensor[filtered_data_to_tensor==0]= float('nan') #didn't find torch.nan function
  

found a suiting amplitude
going to the while loop
elec_i 0,  muscle_i, 0, reps: 0, amp_in_unique_amp, 0.8
finished the while loop
elec_i 0,  muscle_i, 0, reps: 1, amp_in_unique_amp, 0.8
found a suiting amplitude
going to the while loop
elec_i 0,  muscle_i, 0, reps: 0, amp_in_unique_amp, 1.6
finished the while loop
elec_i 0,  muscle_i, 0, reps: 1, amp_in_unique_amp, 1.6
found a suiting amplitude
going to the while loop
elec_i 0,  muscle_i, 0, reps: 0, amp_in_unique_amp, 2.4
going to the while loop
elec_i 0,  muscle_i, 0, reps: 1, amp_in_unique_amp, 2.4
going to the while loop
elec_i 0,  muscle_i, 0, reps: 2, amp_in_unique_amp, 2.4
finished the while loop
elec_i 0,  muscle_i, 0, reps: 3, amp_in_unique_amp, 2.4
found a suiting amplitude
going to the while loop
elec_i 0,  muscle_i, 0, reps: 0, amp_in_unique_amp, 2.5
finished the while loop
elec_i 0,  muscle_i, 0, reps: 1, amp_in_unique_amp, 2.5
found a suiting amplitude
going to the while loop
elec_i 0,  muscle_i, 0, reps: 0, amp_in_unique

## Create peak to peak values tensor

In [36]:
#electrodes
#muscles
#amplitudes
#max_reps
#peak to peak value
max_reps=39 # see notebook 6
peak_to_peak_value=1


peak2peak_data_to_tensor=torch.ones(len(mt_electrodes['elec_name']),len(mt_muscles['muscle_name']),len(unique_amplitudes),max_reps,peak_to_peak_value)
peak2peak_data_to_tensor.size()

torch.Size([16, 16, 40, 39, 1])

In [77]:
## filling the 11 rows (repetition) for the same amp with the raw data/ resp nan value wheb there is no data for that amplitude
for elec_i in range (len(mt_electrodes['elec_name'])) : #loop through all the elecs
    for muscle_i in range (len(mt_muscles['muscle_name'])): # loop through all the muscles
        amp_index=0
        
        for amp_in_unique_amp in unique_amplitudes: # loop through all the amplitudes
            reps=0
            if amp_index< len(filteredData[f"_{elec_i}_{mt_muscles['muscle_name'][muscle_i]}"].Amplitudes):
                #Check for which amplitudes there is data
                if filteredData[f"_{elec_i}_{mt_muscles['muscle_name'][muscle_i]}"].Amplitudes[amp_index]==amp_in_unique_amp:
                    print("found a suiting amplitude")
                    
                    #If only one Rep 
                    if filteredData[f"_{elec_i}_{mt_muscles['muscle_name'][muscle_i]}"].Reps[amp_in_unique_amp]==1:
                        print(f"Only one Rep, thus reps must be zero :{reps} ")

                        peak2peak_data_to_tensor[elec_i, muscle_i, amp_index,reps,:]=torch.tensor(filteredData[f'_{elec_i}_{mt_muscles["muscle_name"][muscle_i]}'].All_peak_peak[amp_in_unique_amp])
                        reps+=1
                        peak2peak_data_to_tensor[elec_i, muscle_i, amp_index,reps:,:]=torch.zeros((len(peak2peak_data_to_tensor[elec_i, muscle_i, amp_index,reps:,0]),peak_to_peak_value)).float()
                        peak2peak_data_to_tensor[peak2peak_data_to_tensor==0]= float('nan') #didn't find torch.nan function
                    elif filteredData[f"_{elec_i}_{mt_muscles['muscle_name'][muscle_i]}"].Reps[amp_in_unique_amp]>1:
                        #Fill all the reps with data
                        while reps<filteredData[f"_{elec_i}_{mt_muscles['muscle_name'][muscle_i]}"].Reps[amp_in_unique_amp]:
                            print("going through the while loop")
                            print(f"elec_i {elec_i},  muscle_i, {muscle_i}, reps: {reps}, amp_in_unique_amp, {amp_in_unique_amp}")
                            peak2peak_data_to_tensor[elec_i, muscle_i, amp_index,reps,0]=torch.tensor(filteredData[f'_{elec_i}_{mt_muscles["muscle_name"][muscle_i]}'].All_peak_peak[amp_in_unique_amp][reps])
                            reps+=1

                    # Fill leftover reps with nan
                    else: 
                        print("finished the while loop")
                        print(f"elec_i {elec_i},  muscle_i, {muscle_i}, reps: {reps}, amp_in_unique_amp, {amp_in_unique_amp}")
                        if filteredData[f'_{elec_i}_{mt_muscles["muscle_name"][muscle_i]}'].Reps[amp_in_unique_amp]<max_reps:
                            peak2peak_data_to_tensor[elec_i, muscle_i, amp_index,reps:,0]=torch.zeros((len(peak2peak_data_to_tensor[elec_i, muscle_i, amp_index,reps:,0]),peak_to_peak_value)).float()
                            peak2peak_data_to_tensor[peak2peak_data_to_tensor==0]= float('nan') #didn't find torch.nan function
                    #increase the amplitude index by one
                    amp_index+=1
                #Fill the amplitudes without data with 'nan'
            else:
                peak2peak_data_to_tensor[elec_i, muscle_i, amp_index,:,:]=torch.zeros((max_reps,peak_to_peak_value)).float()
                peak2peak_data_to_tensor[peak2peak_data_to_tensor==0]= float('nan') #didn't find torch.nan function

found a suiting amplitude
Only one Rep, thus reps must be zero :0 
found a suiting amplitude
Only one Rep, thus reps must be zero :0 
found a suiting amplitude
going through the while loop
elec_i 0,  muscle_i, 0, reps: 0, amp_in_unique_amp, 2.4
going through the while loop
elec_i 0,  muscle_i, 0, reps: 1, amp_in_unique_amp, 2.4
going through the while loop
elec_i 0,  muscle_i, 0, reps: 2, amp_in_unique_amp, 2.4
found a suiting amplitude
Only one Rep, thus reps must be zero :0 
found a suiting amplitude
going through the while loop
elec_i 0,  muscle_i, 0, reps: 0, amp_in_unique_amp, 3.1
going through the while loop
elec_i 0,  muscle_i, 0, reps: 1, amp_in_unique_amp, 3.1
found a suiting amplitude
going through the while loop
elec_i 0,  muscle_i, 0, reps: 0, amp_in_unique_amp, 3.4
going through the while loop
elec_i 0,  muscle_i, 0, reps: 1, amp_in_unique_amp, 3.4
going through the while loop
elec_i 0,  muscle_i, 0, reps: 2, amp_in_unique_amp, 3.4
going through the while loop
elec_i 0,  m

## Create amplitude tensor 

In [82]:

### Create tensor with size torch.Size([16, 14, 32]) 
# nMuslces x nElecs x nAmp_unique 
amplitude_filtereddata_to_tensor=torch.ones(len(mt_electrodes['elec_name']),len(mt_muscles['muscle_name']),len(unique_amplitudes))
amplitude_filtereddata_to_tensor.size()

torch.Size([16, 16, 40])

In [83]:
## newer version, taking repeats into account

## Starting with elec 0, muslce 0, amp=0.5, filling the 10rows (repetition) for the spec. amp with the raw data/ resp nan value since there is no data for that amplitude
for elec_i in range (len(mt_electrodes['elec_name'])) : #loop through all the elecs
    for muscle_i in range (len(mt_muscles['muscle_name'])): # loop through all the muscles
            
        ind=0 # set index to zero, will serve as index from amplitudes
        for pos_in_unique_amplitudes in range(len(unique_amplitudes)): #loop through all the amplitudes possible
            

            if len(filteredData[f"_{elec_i}_{mt_muscles['muscle_name'][muscle_i]}"].Amplitudes) > ind: #Make sure not to exceed size of amplitude count
                data_amp=filteredData[f"_{elec_i}_{mt_muscles['muscle_name'][muscle_i]}"].Amplitudes[ind]
                print(f"Amplitude of unique amp:{unique_amplitudes[pos_in_unique_amplitudes]}, {elec_i}, {mt_muscles['muscle_name'][muscle_i]},  data_amp:{data_amp}")

                if data_amp > unique_amplitudes[pos_in_unique_amplitudes]:
                    print(f"No data for Amplitude: {unique_amplitudes[pos_in_unique_amplitudes]}, next amp with data: {data_amp} fill  nan for this amplitude")
                    amplitude_filtereddata_to_tensor[elec_i, muscle_i, pos_in_unique_amplitudes]=torch.zeros((1)).float()
                    amplitude_filtereddata_to_tensor[amplitude_filtereddata_to_tensor==0]= float('nan') #didn't find torch.nan function
                    
                elif data_amp == unique_amplitudes[pos_in_unique_amplitudes]:
                    #save the amp with data
                    print("same amplitude")
                
                    amplitude_filtereddata_to_tensor[elec_i, muscle_i, pos_in_unique_amplitudes]=torch.Tensor([data_amp])
                    ind+=1
                    print("#################################################################")

            
            else: #No more data for the remaining amplitudes in unique_amplitudes, thus fill with nan
                print(f"In combination {elec_i}_{mt_muscles['muscle_name'][muscle_i]} no more data for {unique_amplitudes[pos_in_unique_amplitudes]}, nan value for this and following amplitudes")
                # amplitude_data_to_tensor[elec_i, muscle_i, pos_in_unique_amplitudes:]=torch.zeros((len(unique_amplitudes)-pos_in_unique_amplitudes)).float()
                amplitude_filtereddata_to_tensor[elec_i, muscle_i, pos_in_unique_amplitudes]=torch.zeros((1)).float()
                amplitude_filtereddata_to_tensor[amplitude_filtereddata_to_tensor==0]= float('nan') #didn't find torch.nan function


Amplitude of unique amp:0.2, 0, LIL,  data_amp:0.8
No data for Amplitude: 0.2, next amp with data: 0.8 fill  nan for this amplitude
Amplitude of unique amp:0.4, 0, LIL,  data_amp:0.8
No data for Amplitude: 0.4, next amp with data: 0.8 fill  nan for this amplitude
Amplitude of unique amp:0.5, 0, LIL,  data_amp:0.8
No data for Amplitude: 0.5, next amp with data: 0.8 fill  nan for this amplitude
Amplitude of unique amp:0.7, 0, LIL,  data_amp:0.8
No data for Amplitude: 0.7, next amp with data: 0.8 fill  nan for this amplitude
Amplitude of unique amp:0.8, 0, LIL,  data_amp:0.8
same amplitude
#################################################################
Amplitude of unique amp:0.9, 0, LIL,  data_amp:1.6
No data for Amplitude: 0.9, next amp with data: 1.6 fill  nan for this amplitude
Amplitude of unique amp:1.0, 0, LIL,  data_amp:1.6
No data for Amplitude: 1.0, next amp with data: 1.6 fill  nan for this amplitude
Amplitude of unique amp:1.1, 0, LIL,  data_amp:1.6
No data for Amplitude: 1.

# Save tensor as pickle file

In [84]:
### save files as pickle file  to projectome_finder#
#to run set run=1
import pickle


file = open('/Users/nealarohner/Desktop/Projectome_Finder/code/Neala_Master_Thesis/data/pickle_files/robin_data_tensor_dump.pkl', "wb")
obj_1 = filtered_data_to_tensor #torch.Size([16, 16, 40, 39, 1000])
obj_2 = amplitude_filtereddata_to_tensor #torch.Size([16, 16, 40])
obj_3 = peak2peak_data_to_tensor #torch.Size([16, 16, 40, 39, 1])

pickle.dump(obj_1, file)
pickle.dump(obj_2, file)
pickle.dump(obj_3, file)

file.close()





## to retrieve ##
# file = open('/Users/nealarohner/Desktop/Projectome_Finder/code/Neala_Master_Thesis/data/pickle_files/robin_data_tensor_dump.pkl', 'rb')
# filtered_data_to_tensor = pickle.load(file)
# amplitude_filtereddata_to_tensor  = pickle.load(file)
# peak2peak_data_to_tensor = pickle.load(file)

# file.close()